In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 37738679
paper_name = 'dolan_fasullo_2023' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheet_names = ['NoP450_400uM_IQ','No_P450_800uM_IQ','Humanized_400uM_IQ','Humanized_800uM_IQ']

In [9]:
original_data_all = []
for sheet_name in sheet_names:
    t = pd.read_excel('raw_data/File S2.xlsx', sheet_name=sheet_name)
    t = t[['Strain','m.value']].set_index('Strain')
    t.columns = [sheet_name]
    print(t.shape)
    original_data_all.append(t)

(5927, 1)
(5927, 1)
(5927, 1)
(5927, 1)


In [10]:
original_data = pd.concat(original_data_all, axis=1)

In [12]:
original_data = original_data.reset_index()

In [13]:
original_data['orf'] = original_data['Strain'].astype(str)

In [14]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [15]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [16]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                 Strain  NoP450_400uM_IQ  No_P450_800uM_IQ  \
index_input                                                  
361          YBR160W_AS         0.097887          0.498823   

             Humanized_400uM_IQ  Humanized_800uM_IQ        orf  
index_input                                                     
361                    1.818383            1.237866  YBR160WAS  


In [17]:
original_data = original_data.loc[t,]

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(5898, 4)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [22260,22261,22262,22263]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,22260,22261,22262,22263
data_type,value,value,value,value
orf,,,,
YAL001C,-3.845008,0.345754,-0.809804,-0.839039
YAL002W,0.099423,0.305412,0.544468,0.325272
YAL003W,0.097887,0.498823,1.818383,1.237866
YAL004W,0.097774,0.260891,0.024210,-0.056975
YAL005C,0.389332,0.604285,0.996898,0.872035


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22260,22261,22262,22263
,data_type,value,value,value,value
gene_id,orf,,,,
1,YAL001C,-3.845008,0.345754,-0.809804,-0.839039
2,YAL002W,0.099423,0.305412,0.544468,0.325272
3,YAL003W,0.097887,0.498823,1.818383,1.237866
1863,YAL004W,0.097774,0.260891,0.024210,-0.056975
4,YAL005C,0.389332,0.604285,0.996898,0.872035


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,22260,22261,22262,22263,22260,22261,22262,22263
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
1,YAL001C,-3.845008,0.345754,-0.809804,-0.839039,-3.780795,0.469165,-0.718497,-0.878183
2,YAL002W,0.099423,0.305412,0.544468,0.325272,0.101922,0.416753,0.381382,0.161683
3,YAL003W,0.097887,0.498823,1.818383,1.237866,0.100409,0.668033,1.415999,0.976737
1863,YAL004W,0.097774,0.260891,0.024210,-0.056975,0.100299,0.358911,-0.041148,-0.179708
4,YAL005C,0.389332,0.604285,0.996898,0.872035,0.387295,0.805049,0.748825,0.650007


# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')